In [1]:
from data_loader import data_importing
training = 'C:/Users/buckf/Documents/Practicum_2/Salt_Deposit_Identification-master/data/train'
testing = 'C:/Users/buckf/Documents/Practicum_2/Salt_Deposit_Identification-master/data/test'

# This Function inputs the training images and masks directory,
# and test images and rezises the images and returns the numpy arrays that are
# resized from 101 x 101 to 128 x 128.
X_train, Y_train = data_importing(path_train=training, path_test=testing)

Using TensorFlow backend.


Getting and resizing train images and masks ... 



Done!


In [2]:
from image_augmentor import aug
train_image_path = r"C:\Users\buckf\Documents\Practicum_2\Salt_Deposit_Identification-master\data\train\images"
train_mask_path = r"C:\Users\buckf\Documents\Practicum_2\Salt_Deposit_Identification-master\data\train\masks"
path_train = r'C:\Users\buckf\Documents\Practicum_2\Salt_Deposit_Identification-master\data\train'
num_images_augmented = 1000


Aug_X_train, Aug_Y_train = aug(train_image_path=train_image_path,
                               train_mask_path=train_mask_path,
                               path_train=path_train,
                              num_images_augmented=num_images_augmented)

Initialised with 4000 image(s) found.
Output directory set to C:\Users\buckf\Documents\Practicum_2\Salt_Deposit_Identification-master\data\train\images\output.

4000 ground truth image(s) found.


Processing <PIL.Image.Image image mode=RGB size=101x101 at 0x1F9964E2C18>: 100%|██████████| 1000/1000 [00:02<00:00, 480.18 Samples/s]


Getting and resizing train images and masks ... 



Done!


In [3]:
import numpy as np
X_train = np.concatenate((X_train, Aug_X_train), axis=0)
Y_train = np.concatenate((Y_train, Aug_Y_train), axis=0)

# Let's use the first 350 images from our trainset for our test sample.
X_test_out_of_train = X_train[:350]
y_test_out_of_train = Y_train[:350]


X_train_w_aug = X_train[350:]
y_train_w_aug = Y_train[350:]

In [4]:
from Modeling import UNET_MODEL

In [5]:
results = UNET_MODEL(X_train_w_aug=X_train_w_aug,
           y_train_w_aug=y_train_w_aug,
          X_test_out_of_train = X_test_out_of_train,
          y_test_out_of_train=y_test_out_of_train)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 1)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 128, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 8)  80          lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 8)  584         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_poolin

Epoch 5/30
 - 18s - loss: 0.3475 - mean_iou: 0.4602 - val_loss: 0.3425 - val_mean_iou: 0.4743

Epoch 00005: val_loss improved from 0.35163 to 0.34249, saving model to model-tgs-salt-1.h5
Epoch 6/30
 - 18s - loss: 0.3265 - mean_iou: 0.4858 - val_loss: 0.3016 - val_mean_iou: 0.4967

Epoch 00006: val_loss improved from 0.34249 to 0.30158, saving model to model-tgs-salt-1.h5
Epoch 7/30
 - 18s - loss: 0.3073 - mean_iou: 0.5063 - val_loss: 0.2853 - val_mean_iou: 0.5157

Epoch 00007: val_loss improved from 0.30158 to 0.28531, saving model to model-tgs-salt-1.h5
Epoch 8/30
 - 18s - loss: 0.2750 - mean_iou: 0.5261 - val_loss: 0.2613 - val_mean_iou: 0.5361

Epoch 00008: val_loss improved from 0.28531 to 0.26129, saving model to model-tgs-salt-1.h5
Epoch 9/30
 - 18s - loss: 0.2620 - mean_iou: 0.5454 - val_loss: 0.2902 - val_mean_iou: 0.5539

Epoch 00009: val_loss did not improve from 0.26129
Epoch 10/30
 - 18s - loss: 0.2470 - mean_iou: 0.5626 - val_loss: 0.2500 - val_mean_iou: 0.5706

Epoch 0001

In [6]:
from data_loader import data_importing_test

In [7]:
# Testing Images
X_test, sizes_test, test_ids = data_importing_test()

Getting and resizing test images ... 



Done!


In [8]:
from keras.models import load_model
from mean_iou import mean_iou
import tqdm
from skimage.transform import resize

# Predict on train, val and test
model = load_model('model-tgs-salt-1.h5', custom_objects={'mean_iou': mean_iou})
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in tqdm.tnrange(len(preds_test)):
    # We need to resize the images from 128 x 128 back to 101 x 101 pixels. 
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

18000/18000 [==============================] - ETA: 3: - ETA: 1: - ETA: 39s - ETA: 30 - ETA: 26 - ETA: 23 - ETA: 21 - ETA: 19 - ETA: 18 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - E

In [10]:
from submission import RLenc
from submission import get_submission
from tqdm import tqdm_notebook as tqdm_NB

pred_dict = {fn[:-4]:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_NB(enumerate(test_ids))}
get_submission(pred_dict)